In [2]:
import numpy as np
import pandas as pd
from sportsreference.ncaab.teams import Teams
from sportsreference.ncaab.teams import Team
from tqdm import tqdm
from sportsreference.ncaab.boxscore import Boxscore
import time

In [13]:
seasons = range(2020, 2021)
master_array = []
for season in tqdm(seasons): 
    try:
        season_team_list = Teams(str(season))
        for team in season_team_list:
            print(team, end = '\r')
            team_schedule = team.schedule
            for game in team_schedule:
                boxscore = game.boxscore_index
                master_array.append([str(season),team.name,boxscore])
    except:
        continue
            
schedule_df = pd.DataFrame(master_array, columns=['Season','TeamName','BoxscoreIndex'])
schedule_df.to_csv('tschedule.csv')

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [02:43<00:00, 163.84s/it]


In [14]:
season = 2020 # will run once for each season
full_box_df = None
counter = 0
season_df = schedule_df.loc[schedule_df.Season==str(season)]
for index, row in tqdm(season_df.iterrows()):
    if counter < np.random.randint(380,520):
        box_link = row['BoxscoreIndex']
        try:
            _df = Boxscore(box_link).dataframe

            if full_box_df is not None:
                box_df = pd.concat([box_df,_df],axis=0)
            else:
                full_box_df = _df
                counter +=1
        except:
            pass
    else:
        time.sleep(np.random.randint(40,90))
        counter = 0
box_df.to_csv('2014_2015_2016_2017_2018_2019_2020_games.csv')

11095it [2:22:39,  1.30it/s]


In [15]:
pd.set_option('display.max_columns', None)
box_df['losing_name'] = box_df['losing_name'].str.replace("\n",'',regex = True)
box_df['losing_name'] = box_df['losing_name'].str.replace("\t",'',regex = True)

,Unnamed: 0,away_assist_percentage,away_assists,away_block_percentage,away_blocks,away_defensive_rating,away_defensive_rebound_percentage,away_defensive_rebounds,away_effective_field_goal_percentage,away_field_goal_attempts,away_field_goal_percentage,away_field_goals,away_free_throw_attempt_rate,away_free_throw_attempts,away_free_throw_percentage,away_free_throws,away_losses,away_minutes_played,away_offensive_rating,away_offensive_rebound_percentage,away_offensive_rebounds,away_personal_fouls,away_points,away_ranking,away_steal_percentage,away_steals,away_three_point_attempt_rate,away_three_point_field_goal_attempts,away_three_point_field_goal_percentage,away_three_point_field_goals,away_total_rebound_percentage,away_total_rebounds,away_true_shooting_percentage,away_turnover_percentage,away_turnovers,away_two_point_field_goal_attempts,away_two_point_field_goal_percentage,away_two_point_field_goals,away_win_percentage,away_wins,date,home_assist_percentage,home_assists,home_block_percentage,home_blocks,home_defensive_rating,home_defensive_rebound_percentage,home_defensive_rebounds,home_effective_field_goal_percentage,home_field_goal_attempts,home_field_goal_percentage,home_field_goals,home_free_throw_attempt_rate,home_free_throw_attempts,home_free_throw_percentage,home_free_throws,home_losses,home_minutes_played,home_offensive_rating,home_offensive_rebound_percentage,home_offensive_rebounds,home_personal_fouls,home_points,home_ranking,home_steal_percentage,home_steals,home_three_point_attempt_rate,home_three_point_field_goal_attempts,home_three_point_field_goal_percentage,home_three_point_field_goals,home_total_rebound_percentage,home_total_rebounds,home_true_shooting_percentage,home_turnover_percentage,home_turnovers,home_two_point_field_goal_attempts,home_two_point_field_goal_percentage,home_two_point_field_goals,home_win_percentage,home_wins,location,losing_abbr,losing_name,pace,winner,winning_abbr,winning_name
0,2013-11-11-st-bonaventure,50.0,7,2.4,1,123.0,43.8,14,0.400,40,0.350,14,0.650,26,0.577,15,2,200,77.0,29.0,9,30,47,NaN,3.3,2,0.400,16,0.250,4,36.5,23,0.449,23.7,16,24,0.417,10,0.000,0,"November 11, 2013",60.0,15,29.2,7,77.0,71.0,22,0.464,56,0.446,25,0.571,32,0.719,23,0,200.0,123.0,56.3,18,23,75,NaN,11.5,7,0.250,14.0,0.143,2.0,63.5,40,0.527,11.4,9,42.0,0.548,23.0,1.000,2,"Reilly Center, St. Bonaventure, New York",ABILENE-CHRISTIAN,Abilene Christian,60.8,Home,ST-BONAVENTURE,St. Bonaventure
1,2013-11-13-maryland,52.9,9,0.0,0,100.0,55.6,15,0.407,54,0.315,17,0.000,0,NaN,0,3,200,65.7,10.3,3,15,44,NaN,9.0,6,0.389,21,0.476,10,32.1,18,0.407,22.9,16,33,0.212,7,0.000,0,"November 13, 2013",46.2,12,30.3,10,65.7,89.7,26,0.528,53,0.491,26,0.321,17,0.647,11,1,200.0,100.0,44.4,12,8,67,NaN,13.4,9,0.340,18.0,0.222,4.0,67.9,38,0.549,21.9,17,35.0,0.629,22.0,0.500,1,"Comcast Center, College Park, Maryland",ABILENE-CHRISTIAN,Abilene Christian,66.5,Home,MARYLAND,Maryland
2,2013-11-17-iowa,42.9,6,0.0,0,145.1,80.0,20,0.287,54,0.259,14,0.222,12,0.833,10,4,200,57.7,18.9,7,21,41,NaN,4.2,3,0.426,23,0.130,3,43.5,27,0.343,22.3,17,31,0.355,11,0.000,0,"November 17, 2013",65.8,25,16.1,5,57.7,81.1,30,0.697,61,0.623,38,0.361,22,0.818,18,0,200.0,145.1,20.0,5,10,103,NaN,15.5,11,0.344,21.0,0.429,9.0,56.5,35,0.721,6.6,5,40.0,0.725,29.0,1.000,4,"Carver-Hawkeye Arena, Iowa City, Iowa",ABILENE-CHRISTIAN,Abilene Christian,70.6,Home,IOWA,Iowa
3,2013-11-19-texas-christian,68.4,13,5.7,2,110.9,76.5,26,0.470,50,0.380,19,0.500,25,0.68,17,5,200,100.0,31.3,10,26,64,NaN,6.3,4,0.400,20,0.450,9,54.5,36,0.517,17.6,13,30,0.333,10,0.000,0,"November 19, 2013",50.0,11,10.0,3,100.0,68.8,22,0.490,48,0.458,22,0.771,37,0.649,24,2,200.0,110.9,23.5,8,20,71,NaN,10.9,7,0.271,13.0,0.231,3.0,45.5,30,0.541,8.5,6,35.0,0.543,19.0,0.333,1,"Daniel-Meyer Coliseum, Fort Worth, Texas",ABILENE-CHRISTIAN,Abilene Christian,64.2,Home,TEXAS-CHRISTIAN,TCU
4,2013-11-22-abilene-christian,NaN,9,NaN,3,NaN,NaN,11,NaN,49,0.347,17,NaN,11,0.636,7,0,200,NaN,NaN,6,22,44,NaN,NaN,12,NaN,13,0.231,3,NaN,17,NaN,NaN,1

In [32]:
box_df = pd.read_csv('2014_2015_2016_2017_2018_2019_2020_games.csv')
box_df.isna().sum().to_frame('nulls').sort_values('nulls',ascending = False).head(20)

,nulls
away_ranking,80678
home_ranking,78420
away_three_point_attempt_rate,1529
away_free_throw_attempt_rate,1529
away_steal_percentage,1529
away_true_shooting_percentage,1529
away_turnover_percentage,1529
away_offensive_rebound_percentage,1529
away_offensive_rating,1529
away_assist_percentage,1529


In [23]:
box_df['away_ranking'] = box_df['home_ranking'].fillna('Unranked')
box_df['away_ranking'] = box_df['home_ranking'].fillna('Unranked')

In [ ]:
Team = 'Indiana'

list_of_stats = ['team','opponent','season','date','pace','result','wins','win_percentage','two_point_field_goals','two_point_field_goal_percentage','two_point_field_goal_attempts','turnovers','turnover_percentage',
            'true_shooting_percentage','total_rebounds','total_rebound_percentage','three_point_field_goals','three_point_field_goal_percentage','three_point_field_goal_attempts',
            'three_point_attempt_rate','steals','steal_percentage','ranking','points','personal_fouls','offensive_rebounds','offensive_rebound_percentage','offensive_rating','minutes_played',
            'losses','free_throws','free_throw_percentage','free_throw_attempts','free_throw_attempt_rate','field_goals','field_goal_percentage','field_goal_attempts','effective_field_goal_percentage',
            'defensive_rebounds','defensive_rating','blocks','block_percentage','assists','assist_percentage']

for stat in list_of_stats[6:]:
    list_of_stats.append('opponent_'+stat)
    
stats_dict = {key:[] for key in list_of_stats}

#Home/Winner
temp_df = box_df[(box_df.winning_name == Team) & (box_df.winner == "Home")]
stats_dict['team'] += (temp_df.winning_name.tolist())
stats_dict['opponent'] += (temp_df.losing_name.tolist())
stats_dict['season'] += ([i[:4] for i in temp_df.index.tolist()])
stats_dict['date'] += (temp_df.date.tolist())
stats_dict['pace'] += (temp_df.pace.tolist())
stats_dict['wins'] += (temp_df.home_wins.tolist())
stats_dict['win_percentage'] += (temp_df.home_win_percentage.tolist())
stats_dict['two_point_field_goals'] += (temp_df.home_two_point_field_goals.tolist())
stats_dict['two_point_field_goal_percentage'] += (temp_df.home_two_point_field_goal_percentage.tolist()) 
stats_dict['two_point_field_goal_attempts'] += (temp_df.home_two_point_field_goal_attempts.tolist())
stats_dict['turnovers'] += (temp_df.home_turnovers.tolist())
stats_dict['turnover_percentage'] += (temp_df.home_turnover_percentage.tolist())
stats_dict['true_shooting_percentage'] += (temp_df.home_true_shooting_percentage.tolist())
stats_dict['total_rebounds'] += (temp_df.home_total_rebounds.tolist())
stats_dict['total_rebound_percentage'] += (temp_df.home_total_rebound_percentage.tolist())
stats_dict['three_point_field_goals'] += (temp_df.home_three_point_field_goals.tolist())
stats_dict['three_point_field_goal_percentage'] += (temp_df.home_three_point_field_goal_percentage.tolist())
stats_dict['three_point_field_goal_attempts'] += (temp_df.home_three_point_field_goal_attempts.tolist())
stats_dict['three_point_attempt_rate'] += (temp_df.home_three_point_attempt_rate.tolist())
stats_dict['steals'] += (temp_df.home_steals.tolist())
stats_dict['steal_percentage'] += (temp_df.home_steal_percentage.tolist())
stats_dict['ranking'] += (temp_df.home_steal_percentage.tolist())
stats_dict['points'] += (temp_df.home_points.tolist())
stats_dict['personal_fouls'] += (temp_df.home_personal_fouls.tolist())
stats_dict['offensive_rebounds'] += (temp_df.home_offensive_rebounds.tolist())
stats_dict['offensive_rebound_percentage'] += (temp_df.home_offensive_rebound_percentage.tolist())
stats_dict['offensive_rating'] += (temp_df.home_offensive_rating.tolist())
stats_dict['minutes_played'] += (temp_df.home_minutes_played.tolist())
stats_dict['losses'] += (temp_df.home_losses.tolist())
stats_dict['free_throws'] += (temp_df.home_free_throws.tolist())
stats_dict['free_throw_percentage'] += (temp_df.home_free_throw_percentage.tolist())
stats_dict['free_throw_attempts'] += (temp_df.home_free_throw_attempts.tolist())
stats_dict['free_throw_attempt_rate'] += (temp_df.home_free_throw_attempt_rate.tolist())
stats_dict['field_goals'] += (temp_df.home_field_goals.tolist())
stats_dict['field_goal_percentage'] += (temp_df.home_field_goal_percentage.tolist())
stats_dict['field_goal_attempts'] += (temp_df.home_field_goal_attempts.tolist())
stats_dict['effective_field_goal_percentage'] += (temp_df.home_effective_field_goal_percentage.tolist())
stats_dict['defensive_rebounds'] += (temp_df.home_defensive_rebounds.tolist())
stats_dict['defensive_rating'] += (temp_df.home_free_throws.tolist())
stats_dict['blocks'] += (temp_df.home_blocks.tolist())
stats_dict['block_percentage'] += (temp_df.home_block_percentage.tolist())
stats_dict['assists'] += (temp_df.home_assists.tolist())
stats_dict['assist_percentage'] += (temp_df.home_assist_percentage.tolist())
stats_dict['result'] += [1] * len(temp_df)
stats_dict['opponent_wins'] += (temp_df.away_wins.tolist())
stats_dict['opponent_win_percentage'] += (temp_df.away_win_percentage.tolist())
stats_dict['opponent_two_point_field_goals'] += (temp_df.away_two_point_field_goals.tolist())
stats_dict['opponent_two_point_field_goal_percentage'] += (temp_df.away_two_point_field_goal_percentage.tolist()) 
stats_dict['opponent_two_point_field_goal_attempts'] += (temp_df.away_two_point_field_goal_attempts.tolist())
stats_dict['opponent_turnovers'] += (temp_df.away_turnovers.tolist())
stats_dict['opponent_turnover_percentage'] += (temp_df.away_turnover_percentage.tolist())
stats_dict['opponent_true_shooting_percentage'] += (temp_df.away_true_shooting_percentage.tolist())
stats_dict['opponent_total_rebounds'] += (temp_df.away_total_rebounds.tolist())
stats_dict['opponent_total_rebound_percentage'] += (temp_df.away_total_rebound_percentage.tolist())
stats_dict['opponent_three_point_field_goals'] += (temp_df.away_three_point_field_goals.tolist())
stats_dict['opponent_three_point_field_goal_percentage'] += (temp_df.away_three_point_field_goal_percentage.tolist())
stats_dict['opponent_three_point_field_goal_attempts'] += (temp_df.away_three_point_field_goal_attempts.tolist())
stats_dict['opponent_three_point_attempt_rate'] += (temp_df.away_three_point_attempt_rate.tolist())
stats_dict['opponent_steals'] += (temp_df.away_steals.tolist())
stats_dict['opponent_steal_percentage'] += (temp_df.away_steal_percentage.tolist())
stats_dict['opponent_ranking'] += (temp_df.away_steal_percentage.tolist())
stats_dict['opponent_points'] += (temp_df.away_points.tolist())
stats_dict['opponent_personal_fouls'] += (temp_df.away_personal_fouls.tolist())
stats_dict['opponent_offensive_rebounds'] += (temp_df.away_offensive_rebounds.tolist())
stats_dict['opponent_offensive_rebound_percentage'] += (temp_df.away_offensive_rebound_percentage.tolist())
stats_dict['opponent_offensive_rating'] += (temp_df.away_offensive_rating.tolist())
stats_dict['opponent_minutes_played'] += (temp_df.away_minutes_played.tolist())
stats_dict['opponent_losses'] += (temp_df.away_losses.tolist())
stats_dict['opponent_free_throws'] += (temp_df.away_free_throws.tolist())
stats_dict['opponent_free_throw_percentage'] += (temp_df.away_free_throw_percentage.tolist())
stats_dict['opponent_free_throw_attempts'] += (temp_df.away_free_throw_attempts.tolist())
stats_dict['opponent_free_throw_attempt_rate'] += (temp_df.away_free_throw_attempt_rate.tolist())
stats_dict['opponent_field_goals'] += (temp_df.away_field_goals.tolist())
stats_dict['opponent_field_goal_percentage'] += (temp_df.away_field_goal_percentage.tolist())
stats_dict['opponent_field_goal_attempts'] += (temp_df.away_field_goal_attempts.tolist())
stats_dict['opponent_effective_field_goal_percentage'] += (temp_df.away_effective_field_goal_percentage.tolist())
stats_dict['opponent_defensive_rebounds'] += (temp_df.away_defensive_rebounds.tolist())
stats_dict['opponent_defensive_rating'] += (temp_df.away_free_throws.tolist())
stats_dict['opponent_blocks'] += (temp_df.away_blocks.tolist())
stats_dict['opponent_block_percentage'] += (temp_df.away_block_percentage.tolist())
stats_dict['opponent_assists'] += (temp_df.away_assists.tolist())
stats_dict['opponent_assist_percentage'] += (temp_df.away_assist_percentage.tolist())

#Away/Winner  Games won on the road
temp_df = box_df[(box_df.winning_name == Team) & (box_df.winner == "Away")]
stats_dict['team'] += (temp_df.winning_name.tolist())
stats_dict['opponent'] += (temp_df.losing_name.tolist())
stats_dict['season'] += ([i[:4] for i in temp_df.index.tolist()])
stats_dict['date'] += (temp_df.date.tolist())
stats_dict['pace'] += (temp_df.pace.tolist())
stats_dict['wins'] += (temp_df.away_wins.tolist())
stats_dict['win_percentage'] += (temp_df.away_win_percentage.tolist())
stats_dict['two_point_field_goals'] += (temp_df.away_two_point_field_goals.tolist())
stats_dict['two_point_field_goal_percentage'] += (temp_df.away_two_point_field_goal_percentage.tolist()) 
stats_dict['two_point_field_goal_attempts'] += (temp_df.away_two_point_field_goal_attempts.tolist())
stats_dict['turnovers'] += (temp_df.away_turnovers.tolist())
stats_dict['turnover_percentage'] += (temp_df.away_turnover_percentage.tolist())
stats_dict['true_shooting_percentage'] += (temp_df.away_true_shooting_percentage.tolist())
stats_dict['total_rebounds'] += (temp_df.away_total_rebounds.tolist())
stats_dict['total_rebound_percentage'] += (temp_df.away_total_rebound_percentage.tolist())
stats_dict['three_point_field_goals'] += (temp_df.away_three_point_field_goals.tolist())
stats_dict['three_point_field_goal_percentage'] += (temp_df.away_three_point_field_goal_percentage.tolist())
stats_dict['three_point_field_goal_attempts'] += (temp_df.away_three_point_field_goal_attempts.tolist())
stats_dict['three_point_attempt_rate'] += (temp_df.away_three_point_attempt_rate.tolist())
stats_dict['steals'] += (temp_df.away_steals.tolist())
stats_dict['steal_percentage'] += (temp_df.away_steal_percentage.tolist())
stats_dict['ranking'] += (temp_df.away_steal_percentage.tolist())
stats_dict['points'] += (temp_df.away_points.tolist())
stats_dict['personal_fouls'] += (temp_df.away_personal_fouls.tolist())
stats_dict['offensive_rebounds'] += (temp_df.away_offensive_rebounds.tolist())
stats_dict['offensive_rebound_percentage'] += (temp_df.away_offensive_rebound_percentage.tolist())
stats_dict['offensive_rating'] += (temp_df.away_offensive_rating.tolist())
stats_dict['minutes_played'] += (temp_df.away_minutes_played.tolist())
stats_dict['losses'] += (temp_df.away_losses.tolist())
stats_dict['free_throws'] += (temp_df.away_free_throws.tolist())
stats_dict['free_throw_percentage'] += (temp_df.away_free_throw_percentage.tolist())
stats_dict['free_throw_attempts'] += (temp_df.away_free_throw_attempts.tolist())
stats_dict['free_throw_attempt_rate'] += (temp_df.away_free_throw_attempt_rate.tolist())
stats_dict['field_goals'] += (temp_df.away_field_goals.tolist())
stats_dict['field_goal_percentage'] += (temp_df.away_field_goal_percentage.tolist())
stats_dict['field_goal_attempts'] += (temp_df.away_field_goal_attempts.tolist())
stats_dict['effective_field_goal_percentage'] += (temp_df.away_effective_field_goal_percentage.tolist())
stats_dict['defensive_rebounds'] += (temp_df.away_defensive_rebounds.tolist())
stats_dict['defensive_rating'] += (temp_df.away_free_throws.tolist())
stats_dict['blocks'] += (temp_df.away_blocks.tolist())
stats_dict['block_percentage'] += (temp_df.away_block_percentage.tolist())
stats_dict['assists'] += (temp_df.away_assists.tolist())
stats_dict['assist_percentage'] += (temp_df.away_assist_percentage.tolist())
stats_dict['result'] += [1] * len(temp_df)
stats_dict['opponent_wins'] += (temp_df.home_wins.tolist())
stats_dict['opponent_win_percentage'] += (temp_df.home_win_percentage.tolist())
stats_dict['opponent_two_point_field_goals'] += (temp_df.home_two_point_field_goals.tolist())
stats_dict['opponent_two_point_field_goal_percentage'] += (temp_df.home_two_point_field_goal_percentage.tolist()) 
stats_dict['opponent_two_point_field_goal_attempts'] += (temp_df.home_two_point_field_goal_attempts.tolist())
stats_dict['opponent_turnovers'] += (temp_df.home_turnovers.tolist())
stats_dict['opponent_turnover_percentage'] += (temp_df.home_turnover_percentage.tolist())
stats_dict['opponent_true_shooting_percentage'] += (temp_df.home_true_shooting_percentage.tolist())
stats_dict['opponent_total_rebounds'] += (temp_df.home_total_rebounds.tolist())
stats_dict['opponent_total_rebound_percentage'] += (temp_df.home_total_rebound_percentage.tolist())
stats_dict['opponent_three_point_field_goals'] += (temp_df.home_three_point_field_goals.tolist())
stats_dict['opponent_three_point_field_goal_percentage'] += (temp_df.home_three_point_field_goal_percentage.tolist())
stats_dict['opponent_three_point_field_goal_attempts'] += (temp_df.home_three_point_field_goal_attempts.tolist())
stats_dict['opponent_three_point_attempt_rate'] += (temp_df.home_three_point_attempt_rate.tolist())
stats_dict['opponent_steals'] += (temp_df.home_steals.tolist())
stats_dict['opponent_steal_percentage'] += (temp_df.home_steal_percentage.tolist())
stats_dict['opponent_ranking'] += (temp_df.home_steal_percentage.tolist())
stats_dict['opponent_points'] += (temp_df.home_points.tolist())
stats_dict['opponent_personal_fouls'] += (temp_df.home_personal_fouls.tolist())
stats_dict['opponent_offensive_rebounds'] += (temp_df.home_offensive_rebounds.tolist())
stats_dict['opponent_offensive_rebound_percentage'] += (temp_df.home_offensive_rebound_percentage.tolist())
stats_dict['opponent_offensive_rating'] += (temp_df.home_offensive_rating.tolist())
stats_dict['opponent_minutes_played'] += (temp_df.home_minutes_played.tolist())
stats_dict['opponent_losses'] += (temp_df.home_losses.tolist())
stats_dict['opponent_free_throws'] += (temp_df.home_free_throws.tolist())
stats_dict['opponent_free_throw_percentage'] += (temp_df.home_free_throw_percentage.tolist())
stats_dict['opponent_free_throw_attempts'] += (temp_df.home_free_throw_attempts.tolist())
stats_dict['opponent_free_throw_attempt_rate'] += (temp_df.home_free_throw_attempt_rate.tolist())
stats_dict['opponent_field_goals'] += (temp_df.home_field_goals.tolist())
stats_dict['opponent_field_goal_percentage'] += (temp_df.home_field_goal_percentage.tolist())
stats_dict['opponent_field_goal_attempts'] += (temp_df.home_field_goal_attempts.tolist())
stats_dict['opponent_effective_field_goal_percentage'] += (temp_df.home_effective_field_goal_percentage.tolist())
stats_dict['opponent_defensive_rebounds'] += (temp_df.home_defensive_rebounds.tolist())
stats_dict['opponent_defensive_rating'] += (temp_df.home_free_throws.tolist())
stats_dict['opponent_blocks'] += (temp_df.home_blocks.tolist())
stats_dict['opponent_block_percentage'] += (temp_df.home_block_percentage.tolist())
stats_dict['opponent_assists'] += (temp_df.home_assists.tolist())
stats_dict['opponent_assist_percentage'] += (temp_df.home_assist_percentage.tolist())

#Away/loser Games lost at home
temp_df = box_df[(box_df.losing_name == Team) & (box_df.winner == "Away")]
stats_dict['team'] += (temp_df.losing_name.tolist())
stats_dict['opponent'] += (temp_df.winning_name.tolist())
stats_dict['season'] += ([i[:4] for i in temp_df.index.tolist()])
stats_dict['date'] += (temp_df.date.tolist())
stats_dict['pace'] += (temp_df.pace.tolist())
stats_dict['wins'] += (temp_df.home_wins.tolist())
stats_dict['win_percentage'] += (temp_df.home_win_percentage.tolist())
stats_dict['two_point_field_goals'] += (temp_df.home_two_point_field_goals.tolist())
stats_dict['two_point_field_goal_percentage'] += (temp_df.home_two_point_field_goal_percentage.tolist()) 
stats_dict['two_point_field_goal_attempts'] += (temp_df.home_two_point_field_goal_attempts.tolist())
stats_dict['turnovers'] += (temp_df.home_turnovers.tolist())
stats_dict['turnover_percentage'] += (temp_df.home_turnover_percentage.tolist())
stats_dict['true_shooting_percentage'] += (temp_df.home_true_shooting_percentage.tolist())
stats_dict['total_rebounds'] += (temp_df.home_total_rebounds.tolist())
stats_dict['total_rebound_percentage'] += (temp_df.home_total_rebound_percentage.tolist())
stats_dict['three_point_field_goals'] += (temp_df.home_three_point_field_goals.tolist())
stats_dict['three_point_field_goal_percentage'] += (temp_df.home_three_point_field_goal_percentage.tolist())
stats_dict['three_point_field_goal_attempts'] += (temp_df.home_three_point_field_goal_attempts.tolist())
stats_dict['three_point_attempt_rate'] += (temp_df.home_three_point_attempt_rate.tolist())
stats_dict['steals'] += (temp_df.home_steals.tolist())
stats_dict['steal_percentage'] += (temp_df.home_steal_percentage.tolist())
stats_dict['ranking'] += (temp_df.home_steal_percentage.tolist())
stats_dict['points'] += (temp_df.home_points.tolist())
stats_dict['personal_fouls'] += (temp_df.home_personal_fouls.tolist())
stats_dict['offensive_rebounds'] += (temp_df.home_offensive_rebounds.tolist())
stats_dict['offensive_rebound_percentage'] += (temp_df.home_offensive_rebound_percentage.tolist())
stats_dict['offensive_rating'] += (temp_df.home_offensive_rating.tolist())
stats_dict['minutes_played'] += (temp_df.home_minutes_played.tolist())
stats_dict['losses'] += (temp_df.home_losses.tolist())
stats_dict['free_throws'] += (temp_df.home_free_throws.tolist())
stats_dict['free_throw_percentage'] += (temp_df.home_free_throw_percentage.tolist())
stats_dict['free_throw_attempts'] += (temp_df.home_free_throw_attempts.tolist())
stats_dict['free_throw_attempt_rate'] += (temp_df.home_free_throw_attempt_rate.tolist())
stats_dict['field_goals'] += (temp_df.home_field_goals.tolist())
stats_dict['field_goal_percentage'] += (temp_df.home_field_goal_percentage.tolist())
stats_dict['field_goal_attempts'] += (temp_df.home_field_goal_attempts.tolist())
stats_dict['effective_field_goal_percentage'] += (temp_df.home_effective_field_goal_percentage.tolist())
stats_dict['defensive_rebounds'] += (temp_df.home_defensive_rebounds.tolist())
stats_dict['defensive_rating'] += (temp_df.home_free_throws.tolist())
stats_dict['blocks'] += (temp_df.home_blocks.tolist())
stats_dict['block_percentage'] += (temp_df.home_block_percentage.tolist())
stats_dict['assists'] += (temp_df.home_assists.tolist())
stats_dict['assist_percentage'] += (temp_df.home_assist_percentage.tolist())
stats_dict['result'] += [0] * len(temp_df)
stats_dict['opponent_wins'] += (temp_df.away_wins.tolist())
stats_dict['opponent_win_percentage'] += (temp_df.away_win_percentage.tolist())
stats_dict['opponent_two_point_field_goals'] += (temp_df.away_two_point_field_goals.tolist())
stats_dict['opponent_two_point_field_goal_percentage'] += (temp_df.away_two_point_field_goal_percentage.tolist()) 
stats_dict['opponent_two_point_field_goal_attempts'] += (temp_df.away_two_point_field_goal_attempts.tolist())
stats_dict['opponent_turnovers'] += (temp_df.away_turnovers.tolist())
stats_dict['opponent_turnover_percentage'] += (temp_df.away_turnover_percentage.tolist())
stats_dict['opponent_true_shooting_percentage'] += (temp_df.away_true_shooting_percentage.tolist())
stats_dict['opponent_total_rebounds'] += (temp_df.away_total_rebounds.tolist())
stats_dict['opponent_total_rebound_percentage'] += (temp_df.away_total_rebound_percentage.tolist())
stats_dict['opponent_three_point_field_goals'] += (temp_df.away_three_point_field_goals.tolist())
stats_dict['opponent_three_point_field_goal_percentage'] += (temp_df.away_three_point_field_goal_percentage.tolist())
stats_dict['opponent_three_point_field_goal_attempts'] += (temp_df.away_three_point_field_goal_attempts.tolist())
stats_dict['opponent_three_point_attempt_rate'] += (temp_df.away_three_point_attempt_rate.tolist())
stats_dict['opponent_steals'] += (temp_df.away_steals.tolist())
stats_dict['opponent_steal_percentage'] += (temp_df.away_steal_percentage.tolist())
stats_dict['opponent_ranking'] += (temp_df.away_steal_percentage.tolist())
stats_dict['opponent_points'] += (temp_df.away_points.tolist())
stats_dict['opponent_personal_fouls'] += (temp_df.away_personal_fouls.tolist())
stats_dict['opponent_offensive_rebounds'] += (temp_df.away_offensive_rebounds.tolist())
stats_dict['opponent_offensive_rebound_percentage'] += (temp_df.away_offensive_rebound_percentage.tolist())
stats_dict['opponent_offensive_rating'] += (temp_df.away_offensive_rating.tolist())
stats_dict['opponent_minutes_played'] += (temp_df.away_minutes_played.tolist())
stats_dict['opponent_losses'] += (temp_df.away_losses.tolist())
stats_dict['opponent_free_throws'] += (temp_df.away_free_throws.tolist())
stats_dict['opponent_free_throw_percentage'] += (temp_df.away_free_throw_percentage.tolist())
stats_dict['opponent_free_throw_attempts'] += (temp_df.away_free_throw_attempts.tolist())
stats_dict['opponent_free_throw_attempt_rate'] += (temp_df.away_free_throw_attempt_rate.tolist())
stats_dict['opponent_field_goals'] += (temp_df.away_field_goals.tolist())
stats_dict['opponent_field_goal_percentage'] += (temp_df.away_field_goal_percentage.tolist())
stats_dict['opponent_field_goal_attempts'] += (temp_df.away_field_goal_attempts.tolist())
stats_dict['opponent_effective_field_goal_percentage'] += (temp_df.away_effective_field_goal_percentage.tolist())
stats_dict['opponent_defensive_rebounds'] += (temp_df.away_defensive_rebounds.tolist())
stats_dict['opponent_defensive_rating'] += (temp_df.away_free_throws.tolist())
stats_dict['opponent_blocks'] += (temp_df.away_blocks.tolist())
stats_dict['opponent_block_percentage'] += (temp_df.away_block_percentage.tolist())
stats_dict['opponent_assists'] += (temp_df.away_assists.tolist())
stats_dict['opponent_assist_percentage'] += (temp_df.away_assist_percentage.tolist())

#Home/loser Games lost on the road
temp_df = box_df[(box_df.losing_name == Team) & (box_df.winner == "Home")]
stats_dict['team'] += (temp_df.losing_name.tolist())
stats_dict['opponent'] += (temp_df.winning_name.tolist())
stats_dict['season'] += ([i[:4] for i in temp_df.index.tolist()])
stats_dict['date'] += (temp_df.date.tolist())
stats_dict['pace'] += (temp_df.pace.tolist())
stats_dict['wins'] += (temp_df.away_wins.tolist())
stats_dict['win_percentage'] += (temp_df.away_win_percentage.tolist())
stats_dict['two_point_field_goals'] += (temp_df.away_two_point_field_goals.tolist())
stats_dict['two_point_field_goal_percentage'] += (temp_df.away_two_point_field_goal_percentage.tolist()) 
stats_dict['two_point_field_goal_attempts'] += (temp_df.away_two_point_field_goal_attempts.tolist())
stats_dict['turnovers'] += (temp_df.away_turnovers.tolist())
stats_dict['turnover_percentage'] += (temp_df.away_turnover_percentage.tolist())
stats_dict['true_shooting_percentage'] += (temp_df.away_true_shooting_percentage.tolist())
stats_dict['total_rebounds'] += (temp_df.away_total_rebounds.tolist())
stats_dict['total_rebound_percentage'] += (temp_df.away_total_rebound_percentage.tolist())
stats_dict['three_point_field_goals'] += (temp_df.away_three_point_field_goals.tolist())
stats_dict['three_point_field_goal_percentage'] += (temp_df.away_three_point_field_goal_percentage.tolist())
stats_dict['three_point_field_goal_attempts'] += (temp_df.away_three_point_field_goal_attempts.tolist())
stats_dict['three_point_attempt_rate'] += (temp_df.away_three_point_attempt_rate.tolist())
stats_dict['steals'] += (temp_df.away_steals.tolist())
stats_dict['steal_percentage'] += (temp_df.away_steal_percentage.tolist())
stats_dict['ranking'] += (temp_df.away_steal_percentage.tolist())
stats_dict['points'] += (temp_df.away_points.tolist())
stats_dict['personal_fouls'] += (temp_df.away_personal_fouls.tolist())
stats_dict['offensive_rebounds'] += (temp_df.away_offensive_rebounds.tolist())
stats_dict['offensive_rebound_percentage'] += (temp_df.away_offensive_rebound_percentage.tolist())
stats_dict['offensive_rating'] += (temp_df.away_offensive_rating.tolist())
stats_dict['minutes_played'] += (temp_df.away_minutes_played.tolist())
stats_dict['losses'] += (temp_df.away_losses.tolist())
stats_dict['free_throws'] += (temp_df.away_free_throws.tolist())
stats_dict['free_throw_percentage'] += (temp_df.away_free_throw_percentage.tolist())
stats_dict['free_throw_attempts'] += (temp_df.away_free_throw_attempts.tolist())
stats_dict['free_throw_attempt_rate'] += (temp_df.away_free_throw_attempt_rate.tolist())
stats_dict['field_goals'] += (temp_df.away_field_goals.tolist())
stats_dict['field_goal_percentage'] += (temp_df.away_field_goal_percentage.tolist())
stats_dict['field_goal_attempts'] += (temp_df.away_field_goal_attempts.tolist())
stats_dict['effective_field_goal_percentage'] += (temp_df.away_effective_field_goal_percentage.tolist())
stats_dict['defensive_rebounds'] += (temp_df.away_defensive_rebounds.tolist())
stats_dict['defensive_rating'] += (temp_df.away_free_throws.tolist())
stats_dict['blocks'] += (temp_df.away_blocks.tolist())
stats_dict['block_percentage'] += (temp_df.away_block_percentage.tolist())
stats_dict['assists'] += (temp_df.away_assists.tolist())
stats_dict['assist_percentage'] += (temp_df.away_assist_percentage.tolist())
stats_dict['result'] += [0] * len(temp_df)
stats_dict['opponent_wins'] += (temp_df.home_wins.tolist())
stats_dict['opponent_win_percentage'] += (temp_df.home_win_percentage.tolist())
stats_dict['opponent_two_point_field_goals'] += (temp_df.home_two_point_field_goals.tolist())
stats_dict['opponent_two_point_field_goal_percentage'] += (temp_df.home_two_point_field_goal_percentage.tolist()) 
stats_dict['opponent_two_point_field_goal_attempts'] += (temp_df.home_two_point_field_goal_attempts.tolist())
stats_dict['opponent_turnovers'] += (temp_df.home_turnovers.tolist())
stats_dict['opponent_turnover_percentage'] += (temp_df.home_turnover_percentage.tolist())
stats_dict['opponent_true_shooting_percentage'] += (temp_df.home_true_shooting_percentage.tolist())
stats_dict['opponent_total_rebounds'] += (temp_df.home_total_rebounds.tolist())
stats_dict['opponent_total_rebound_percentage'] += (temp_df.home_total_rebound_percentage.tolist())
stats_dict['opponent_three_point_field_goals'] += (temp_df.home_three_point_field_goals.tolist())
stats_dict['opponent_three_point_field_goal_percentage'] += (temp_df.home_three_point_field_goal_percentage.tolist())
stats_dict['opponent_three_point_field_goal_attempts'] += (temp_df.home_three_point_field_goal_attempts.tolist())
stats_dict['opponent_three_point_attempt_rate'] += (temp_df.home_three_point_attempt_rate.tolist())
stats_dict['opponent_steals'] += (temp_df.home_steals.tolist())
stats_dict['opponent_steal_percentage'] += (temp_df.home_steal_percentage.tolist())
stats_dict['opponent_ranking'] += (temp_df.home_steal_percentage.tolist())
stats_dict['opponent_points'] += (temp_df.home_points.tolist())
stats_dict['opponent_personal_fouls'] += (temp_df.home_personal_fouls.tolist())
stats_dict['opponent_offensive_rebounds'] += (temp_df.home_offensive_rebounds.tolist())
stats_dict['opponent_offensive_rebound_percentage'] += (temp_df.home_offensive_rebound_percentage.tolist())
stats_dict['opponent_offensive_rating'] += (temp_df.home_offensive_rating.tolist())
stats_dict['opponent_minutes_played'] += (temp_df.home_minutes_played.tolist())
stats_dict['opponent_losses'] += (temp_df.home_losses.tolist())
stats_dict['opponent_free_throws'] += (temp_df.home_free_throws.tolist())
stats_dict['opponent_free_throw_percentage'] += (temp_df.home_free_throw_percentage.tolist())
stats_dict['opponent_free_throw_attempts'] += (temp_df.home_free_throw_attempts.tolist())
stats_dict['opponent_free_throw_attempt_rate'] += (temp_df.home_free_throw_attempt_rate.tolist())
stats_dict['opponent_field_goals'] += (temp_df.home_field_goals.tolist())
stats_dict['opponent_field_goal_percentage'] += (temp_df.home_field_goal_percentage.tolist())
stats_dict['opponent_field_goal_attempts'] += (temp_df.home_field_goal_attempts.tolist())
stats_dict['opponent_effective_field_goal_percentage'] += (temp_df.home_effective_field_goal_percentage.tolist())
stats_dict['opponent_defensive_rebounds'] += (temp_df.home_defensive_rebounds.tolist())
stats_dict['opponent_defensive_rating'] += (temp_df.home_free_throws.tolist())
stats_dict['opponent_blocks'] += (temp_df.home_blocks.tolist())
stats_dict['opponent_block_percentage'] += (temp_df.home_block_percentage.tolist())
stats_dict['opponent_assists'] += (temp_df.home_assists.tolist())
stats_dict['opponent_assist_percentage'] += (temp_df.home_assist_percentage.tolist())

In [ ]:
final_df = pd.DataFrame.from_dict(stats_dict)
final_df['date'] = pd.to_datetime(final_df['date'])
final_df.drop_duplicates(inplace = True)
final_df.sort_values('date')